In [1]:
import numpy as np
import pandas as pd
import requests
import urllib3
import json
import time
from datetime import datetime, timedelta

In [2]:
SPY = 756733

In [3]:
urllib3.disable_warnings()

def get(url, params=None):
    return requests.get(url, params=params, verify=False).json()

def post(url, data=None):
    return requests.post(url, data, verify=False)

def validate():
    return post("https://localhost:5000/v1/api/sso/validate")

def status():
    return post("https://localhost:5000/v1/api/iserver/auth/status")

def reauthenticate():
    return post("https://localhost:5000/v1/api/iserver/reauthenticate")

def logout():
    return post("https://localhost:5000/v1/api/logout")

In [4]:
def wait_after_next_minute(minutes=1, seconds=0):
    current_time = datetime.now()
    new_time = current_time.replace(second=0, microsecond=0)
    uptime = new_time + timedelta(minutes=1, seconds=5)
    delay = uptime - current_time
    time.sleep(delay.total_seconds())

In [5]:
def get_bars(conid, period, bar, outsideRth=False):
    payload = {"conid" : conid, "period" : period, "bar" : bar, "outsideRth":outsideRth}
    r = get("https://localhost:5000/v1/api/iserver/marketdata/history", params=payload)
    bars = r['data']
    return bars

def get_dataframe_bars(conid, period, bar, outsideRth=False):
    bars = get_bars(conid, period, bar, outsideRth)
    df = pd.DataFrame(bars)
    df.set_index('t', inplace=True)
    return df
    
                    

In [6]:
get_bars(SPY, '1min', '1min')

[{'o': 474.9,
  'c': 474.88,
  'h': 475.01,
  'l': 474.67,
  'v': 27253,
  't': 1640984340000}]

TypeError: an integer is required (got type datetime.datetime)

In [8]:
datetime.utcnow()

datetime.datetime(2022, 1, 2, 10, 18, 15, 193875)

In [12]:
status().json()

{'authenticated': True,
 'competing': False,
 'connected': True,
 'message': '',
 'MAC': 'F4:03:43:DC:90:80',
 'fail': ''}

In [ ]:
wait_after_next_minute()
bars = get_bars(SPY, '10min', '1min')
bars.pop()

In [ ]:
for i in range(5):
    wait_after_next_minute()
    b = get_bars(SPY, '1min', '1min')
    print()
    for i,x in enumerate(b):
        print(i,x)
    bars.append(b[0])

In [ ]:
df = pd.DataFrame(bars)
df